In [15]:
import pandas as pd
import numpy as np

In [16]:
def payer_category(payer):
    if 'GrossCharge' in payer:
        return 'gross'
    elif 'DiscountedCashPrice' in payer or 'SELFPAY' in payer:
        return 'cash'
    elif 'Minimun'in payer:
        return'min'
    elif 'Maximun' in payer:
        return'max'
    else:
        return 'payer'

def patient_class(payer):
    if 'Inpatient' in payer or '_IPPS' in payer:
        return 'inpatient'
    elif 'Outpatient' in payer or '_OPPS' in payer:
        return 'outpatient'
    elif 'Emergency' in payer:
        return 'emergency'
    
def rate_desc(payer):
    if 'PercentOfCharges' in payer:
        return 'PercentOfCharges'
    elif 'TieredFeeSchedule' in payer:
        return 'TieredFeeSchedule'

In [17]:
file = '640468873_southwestmississippiregionalmedicalcenter_standardcharges.csv'

df = pd.read_csv('.\\input_files\\' + file, dtype={'MSDRG': str, 'NDC': str})

\Local\Temp\ipykernel_28580\1471174646.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('.\\input_files\\' + file, dtype={'MSDRG': str, 'NDC': str})


In [18]:
# Skip first 3 rows
df = df[3:]

In [19]:
df = df.rename(columns={
    'ProcedureCode': 'code',
    'Modifier': 'modifier',
    'RevenueCode': 'rev_code',
    'Description': 'description'
})

In [20]:
# Merge column `MSDRG` and `code`
df.loc[df['code'].notna(), 'code_prefix'] = 'hcpcs_cpt'
df['code'] = df['code'].fillna(df['MSDRG'])
# This is fine because any null code prefix will either be ms-drg 
# or have a null `code` and will thus be dropped.
df['code_prefix'] = df['code_prefix'].fillna('ms-drg')
df.drop(columns=['MSDRG', 'RecordType', 'FileInformation'], inplace=True)

In [21]:
# Bring added columns to the beginning to make the next step easier (reorder)
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

In [22]:
cols = df.columns.tolist()
split_index = cols.index('InpatientGrossCharge')

id_vars = cols[:split_index]
value_vars =  cols[split_index:]

df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer_orig', value_name='rate')

In [23]:
df.dropna(subset=['rate', 'code'], inplace=True)

In [24]:
df['payer_category'] = df['payer_orig'].apply(payer_category)
df['patient_class'] = df['payer_orig'].apply(patient_class)
df['rate_desc'] = df['payer_orig'].apply(rate_desc)

In [25]:
df

,code_prefix,description,code,modifier,rev_code,NDC,payer_orig,rate,payer_category,patient_class,rate_desc
294,hcpcs_cpt,ADMIN PFIZER COVID-19 VACCINE 1ST DOSE,0001A,NaN,NaN,NaN,InpatientGrossCharge,90.00,gross,inpatient,None
295,hcpcs_cpt,ADMIN PFIZER COVID-19 VACCINE 2ND DOSE,0002A,NaN,NaN,NaN,InpatientGrossCharge,90.00,gross,inpatient,None
296,hcpcs_cpt,ADMIN PFIZER COVID-19 VACCINE 3RD DOSE,0003A,NaN,NaN,NaN,InpatientGrossCharge,90.00,gross,inpatient,None
297,hcpcs_cpt,ADMIN PFIZER COVID-19 VACCINE BOOSTER,0004A,NaN,NaN,NaN,InpatientGrossCharge,90.00,gross,inpatient,None
298,hcpcs_cpt,ADMIN MODERNA COVID-19 VACCINE 1ST DOSE,0011A,NaN,NaN,NaN,InpatientGrossCharge,90.00,gross,inpatient,None
...,...,...,...,...,...,...,...,...,...,...,...
1497465,ms-drg,HIV WITH MAJOR RELATED CONDITION WITH MCC,974,NaN,NaN,NaN,WINDSORMCAREEXTRA_IPPS,18322.42,payer,inpatient,None
1497466,ms-drg,HIV WITH MAJOR RELATED CONDITION WITHOUT CC/MCC,976,NaN,NaN,NaN,WINDSORMCAREEXTRA_IPPS,6423.13,payer,inpatient,None
1497467,ms-drg,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,981,NaN,NaN,NaN,WINDSORMCAREEXTRA_IPPS,28740.25,payer,inpatient,None
1497468,ms-drg,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,982,NaN,NaN,NaN,WINDSORMCAREEXTRA_IPPS,16096.57,payer,inpatient,None


In [27]:
# Update rate_desc based on conditions
df.loc[(df["payer_category"] == "gross") & (df["code_prefix"] == "ms-drg"), "rate_desc"] = "averaged rate"
df = df[~((df['patient_class'] == 'outpatient') & (df['code_prefix'] == 'ms-drg'))]

In [120]:
df['file_last_updated'] = '2023-02-01'
df['url'] = 'https://pricetransparency.blob.core.windows.net/smrmc/640468873_southwestmississippiregionalmedicalcenter_standardcharges.csv'
df['filename'] = file
df['hospital_ein'] = '64-0468873'
df['hospital_ccn'] = '250097'

df['payer_name'] = df['payer_orig']
df['code_orig'] = df['code']

In [121]:
df

,code_prefix,description,code,modifier,rev_code,NDC,payer_orig,rate,payer_category,patient_class,rate_desc,file_last_updated,url,filename,hospital_ein,hospital_ccn,payer_name,code_orig
294,hcpcs_cpt,ADMIN PFIZER COVID-19 VACCINE 1ST DOSE,0001A,NaN,NaN,NaN,InpatientGrossCharge,90.00,gross,inpatient,None,2023-02-01,https://pricetransparency.blob.core.windows.ne...,640468873_southwestmississippiregionalmedicalc...,64-0468873,250097,InpatientGrossCharge,0001A
295,hcpcs_cpt,ADMIN PFIZER COVID-19 VACCINE 2ND DOSE,0002A,NaN,NaN,NaN,InpatientGrossCharge,90.00,gross,inpatient,None,2023-02-01,https://pricetransparency.blob.core.windows.ne...,640468873_southwestmississippiregionalmedicalc...,64-0468873,250097,InpatientGrossCharge,0002A
296,hcpcs_cpt,ADMIN PFIZER COVID-19 VACCINE 3RD DOSE,0003A,NaN,NaN,NaN,InpatientGrossCharge,90.00,gross,inpatient,None,2023-02-01,https://pricetransparency.blob.core.windows.ne...,640468873_southwestmississippiregionalmedicalc...,64-0468873,250097,InpatientGrossCharge,0003A
297,hcpcs_cpt,ADMIN PFIZER COVID-19 VACCINE BOOSTER,0004A,NaN,NaN,NaN,InpatientGrossCharge,90.00,gross,inpatient,None,2023-02-01,https://pricetransparency.blob.core.windows.ne...,640468873_southwestmississippiregionalmedicalc...,64-0468873,250097,InpatientGrossCharge,0004A
298,hcpcs_cpt,ADMIN MODERNA COVID-19 VACCINE 1ST DOSE,0011A,NaN,NaN,NaN,InpatientGrossCharge,90.00,gross,inpatient,None,2023-02-01,https://pricetransparency.blob.core.windows.ne...,640468873_southwestmississippiregionalmedicalc...,64-0468873,250097,InpatientGrossCharge,0011A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1497465,ms-drg,HIV WITH MAJOR RELATED CONDITION WITH MCC,974,NaN,NaN,NaN,WINDSORMCAREEXTRA_IPPS,18322.42,payer,inpatient,None,2023-02-01,https://pricetransparency.blob.core.windows.ne...,640468873_southwestmississippiregionalmedicalc...,64-0468873,250097,WINDSORMCAREEXTRA_IPPS,974
1497466,ms-drg,HIV WITH MAJOR RELATED CONDITION WITHOUT CC/MCC,976,NaN,NaN,NaN,WINDSORMCAREEXTRA_IPPS,6423.13,payer,inpatient,None,2023-02-01,https://pricetransparency.blob.core.windows.ne...,640468873_southwestmississippiregionalmedicalc...,64-0468873,250097,WINDSORMCAREEXTRA_IPPS,976
1497467,ms-drg,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,981,NaN,NaN,NaN,WINDSORMCAREEXTRA_IPPS,28740.25,payer,inpatient,None,2023-02-01,https://pricetransparency.blob.core.windows.ne...,640468873_southwestmississippiregionalmedicalc...,64-0468873,250097,WINDSORMCAREEXTRA_IPPS,981
1497468,ms-drg,EXTENSIVE O.R. PROCEDURES UNRELATED TO PRINCIP...,982,NaN,NaN,NaN,WINDSORMCAREEXTRA_IPPS,16096.57,payer,inpatient,None,2023-02-01,https://pricetransparency.blob.core.windows.ne...,640468873_southwestmississippiregionalmedicalc...,64-0468873,250097,WINDSORMCAREEXTRA_IPPS,982


In [110]:
# # Select rows of df where 'code_prefix` = 'ms-drg` and patient_class = 'outpatient' using np.where()
# mask1 = df['code_prefix'] == 'ms-drg'
# mask2 = df['patient_class'] == 'outpatient'

# # Use boolean masks with & operator for element-wise comparison
# result = df[mask1 & mask2]
# result

In [105]:
a

(array([     0,      1,      2, ..., 921370, 921371, 921372], dtype=int64),)